# Few Shot in Context Learning¶

In [254]:
import os
import json
import csv
import time
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from bs4 import BeautifulSoup
import requests
from langchain.prompts.chat import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_mistralai import ChatMistralAI
from langchain.llms import Ollama
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.callbacks.base import BaseCallbackHandler
from langchain.tools.base import Tool  
from typing import Callable

In [256]:
############### CONFIG ###################

# Load configuration from JSON (mistral, openai, ollama, anthropic, google)
CONFIG_FILE = "config-no-API-key/llm_config_openai.json"
MODELS_FILE = "config-no-API-key/llm_models.json"

In [258]:
########### FUNCTIONS ##########
def load_config(config_file):
    try:
        with open(config_file, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Configuration file {config_file} not found.")
        return {}
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return {}

def extract_text_from_pdf(pdf_path):
    """
    Estrae e restituisce il testo dal file PDF specificato.
    """
    text = ""
    try:
        with open(pdf_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text
    except Exception as e:
        print(f"Errore nell'estrazione del testo da {pdf_path}: {e}")
    return text

def save_to_file(file_path, content):
    with open(file_path, 'w') as file:
        file.write(content)

def save_metadata(file_path, metadata):
    with open(file_path, 'w') as file:
        json.dump(metadata, file, indent=4)

In [260]:
# Load configurations
config = load_config(CONFIG_FILE)
models_config = load_config(MODELS_FILE)

In [262]:
# Extract parameters from configuration
LLM = config.get("llm")
if not LLM:
    raise ValueError("LLM name must be specified in the configuration file.")

PRICE_PER_INPUT_TOKEN = config.get("price_per_input_token")
PRICE_PER_OUTPUT_TOKEN = config.get("price_per_output_token")
temperature = config.get("temperature")
max_retries = config.get("max_retries")
api_key = config.get("api_keys", {}).get(LLM.lower(), None)
base_url = config.get("base_url")

# Get model configuration
LLM_TYPE = 'Other'
llm_config = models_config.get(LLM, None)
if llm_config and LLM_TYPE != 'Ollama':
    # Update parameters dynamically
    llm_params = llm_config.get("params", {})
    llm_params["temperature"] = temperature
    llm_params["max_retries"] = max_retries
    llm_params["api_key"] = api_key
    llm_params["base_url"] = base_url

    # Initialize LLM
    llm_class = eval(llm_config["class"])
    llm_LangChain = llm_class(**llm_params)
    model_name = LLM  # Use LLM directly as the model name
elif LLM_TYPE == 'Ollama':
    llm_params = llm_config.get("params", {})
    llm_params["temperature"] = temperature
    llm_params["base_url"] = base_url

    # Initialize LLM
    llm_class = eval(llm_config["class"])
    llm_LangChain = llm_class(**llm_params)
    model_name = LLM  # Use LLM directly as the model name
else:
    raise ValueError(f"Model configuration for '{LLM}' not found in {MODELS_FILE}.")

In [264]:
import os
import PyPDF2

# Few-shot context
context = (
    "I due testi che sto per farti analizzare sono uguali. Uno in lingua inglese e uno in lingua latina."
)

task = (
    "Esegui named entity recognition ed estrapola anche i luoghi intesi come borghi, navi, carceri, mercati e luoghi similari."
    "Estrapola anche oggetti."
)

base_output_dir = f"Prompt1-Results-{model_name.lower()}-{temperature}"
base_output_json_dir = f"Prompt1-Results-{model_name.lower()}-{temperature}/JSON"

# Ensure output directory exists
os.makedirs(base_output_dir, exist_ok=True)
os.makedirs(base_output_json_dir, exist_ok=True)
    
# Specifica la cartella in cui cercare i file PDF
folder_path = "doc"  # <-- Sostituisci con il percorso corretto

# Dizionario per salvare le coppie di file: la chiave è il nome base, 
# e il valore è un dizionario con le chiavi 'english' e 'latin'
pairs = {}

# Scorri tutti i file nella cartella
for file in os.listdir(folder_path):
    if file.lower().endswith(".pdf"):
        # Se il file termina con "latino.pdf" è la versione in latino
        if file.lower().endswith("latino.pdf"):
            # Rimuovo gli ultimi 10 caratteri ("latino.pdf") per ottenere il nome base
            base_name = file[:-10]
            if base_name not in pairs:
                pairs[base_name] = {}
            pairs[base_name]['latin'] = os.path.join(folder_path, file)
        else:
            # Altrimenti, supponiamo sia la versione in inglese: rimuovo ".pdf"
            base_name = file[:-4]
            if base_name not in pairs:
                pairs[base_name] = {}
            pairs[base_name]['english'] = os.path.join(folder_path, file)

for base, files in pairs.items():
    # Verifica che esista la coppia (versione in inglese e versione in latino)
    if 'english' in files and 'latin' in files:
        english_text = extract_text_from_pdf(files['english'])
        latin_text   = extract_text_from_pdf(files['latin'])

        print(files['english'])

        ############ Save file ##########
        output_res_path = os.path.join(base_output_dir, os.path.basename(files['english']) + '.txt')
        print(output_res_path)
        metadata_path = os.path.join(base_output_json_dir, os.path.basename(files['english']) + '.json')
        print(metadata_path)

        ####################### Synthetic Modeling Operation ###########################
        # Chat few-shot prompt template
        chat_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "Sei un assistente esperto in Named Entity Recognition e analisi testuale, con una profonda conoscenza delle lingue moderna e latina."),
                (
                    "user",
                    f"Context:\n{context}\n\n"
                    "Testo in inglese: {english_text}"
                    "Testo in latino: {latin_text}"
                    f"Task:\n{task}\n\n"
                )
            ]
        )

        print(chat_prompt)

        # Start timer
        start_time = time.time()
        
        # Invoke the model
        response = chat_prompt | llm_LangChain
        result = response.invoke(
            {
                "english_text": english_text,
                "latin_text": latin_text
            }
        )

        # Stop timer
        end_time = time.time()
        execution_time = end_time - start_time  # Prompt Execution Time (in second)

        if LLM_TYPE != 'Ollama':
            output_res = result.content.strip()
        else:
            output_res = result.strip()
        print(result)
        
        if LLM_TYPE != 'Ollama':
            metadata = {
                "response_length": len(output_res),
                "execution_time": execution_time,
                "temperature": temperature,
                "usage": result.usage_metadata,
                "price_usd": result.usage_metadata.get("input_tokens", 0) * PRICE_PER_INPUT_TOKEN + result.usage_metadata.get("output_tokens", 0) * PRICE_PER_OUTPUT_TOKEN,
                "model_name": model_name
            }
        else:
            metadata = {
                "response_length": len(output_res),
                "execution_time": execution_time,
                "temperature": temperature,
                "model_name": model_name
            }

        save_to_file(output_res_path, output_res)
        save_metadata(metadata_path, metadata)

        print(f"Processed: {files['english']}")
        print(f"Results saved to: {base_output_dir}")
        print(f"Metadata saved to: {base_output_json_dir}")

    else:
        print(f"Attenzione: La coppia per '{base}' non è completa. File trovati: {list(files.keys())}")


doc\doc11.pdf
Prompt1-Results-gpt-4o-2024-08-06-0.7\doc11.pdf.txt
Prompt1-Results-gpt-4o-2024-08-06-0.7/JSON\doc11.pdf.json
input_variables=['english_text', 'latin_text'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Sei un assistente esperto in Named Entity Recognition e analisi testuale, con una profonda conoscenza delle lingue moderna e latina.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['english_text', 'latin_text'], input_types={}, partial_variables={}, template='Context:\nI due testi che sto per farti analizzare sono uguali. Uno in lingua inglese e uno in lingua latina.\n\nTesto in inglese: {english_text}Testo in latino: {latin_text}Task:\nEsegui named entity recognition ed estrapola anche i luoghi intesi come borghi, navi, carceri, mercati e luoghi similari.Estrapola anche oggetti.\n\n'), additional_kwargs={})]
cont